In [1]:
#Set dependencies
import pandas as pd
import os
import numpy as np
from functools import reduce

sensors = ['42019','42043','42044','42047','8764314','8770613', 'TABS-X']
#sensors = ['42047']

months = ['June 2017','July 2017','August 2017','Sept 2017']
#months = ['Sept 2017']

monthsf = ['0617','0717','0817','0917']
#monthsf = ['0917']

file1 = []
file2 = []
file3 = []
file4 = []
file5 = []

#naming csv file for all resources
for i in sensors:
    for j in months:
        file1.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} air_pressure.csv")
        file2.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} air_temperature.csv")
        file3.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} sea_water_practical_salinity.csv")
        file4.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} sea_water_temperature.csv")
        file5.append(f"../Resources/Project Data/Dirty Trail Data/Harvey 17/{i}/{j} winds.csv")
        
print(len(file1),len(file2),len(file3),len(file4),len(file5))

28 28 28 28 28


In [2]:
# Read all csv files
files = []
for i in range(len(sensors)*len(months)):
        #print(f'Reading csvfile {i}')
        df1 = pd.read_csv(file1[i], parse_dates=['date'])
        df2 = pd.read_csv(file2[i], parse_dates=['date'])
        df3 = pd.read_csv(file3[i], parse_dates=['date'])
        df4 = pd.read_csv(file4[i], parse_dates=['date'])
        df5 = pd.read_csv(file5[i], parse_dates=['date'])
        
        print(len(df1),len(df2),len(df3),len(df4),len(df5))
        
        df1 = df1.set_index('date')
        df2 = df2.set_index('date')
        df3 = df3.set_index('date')
        df4 = df4.set_index('date')
        df5 = df5.set_index('date')
        
        
        dfa = [df1,df2,df3,df4,df5]
        files.append(dfa)
        
dfx = pd.DataFrame(files) 
dfx.shape

618 618 1 594 618
729 728 1 671 728
630 628 1 547 629
665 665 1 635 666
1153 1153 764 1153 1154
728 728 1 728 729
452 452 441 456 453
1441 1441 1441 1441 1442
1441 1441 1441 1441 1442
1486 1486 1486 1486 1487
1489 1489 1489 1489 1490
1441 1441 1441 1441 1442
1441 1441 1441 1441 1442
1488 1488 1488 1488 1489
1488 1488 1488 466 1489
1441 1441 1441 1 1442
2557 1548 1887 1703 624
3036 1916 2234 1941 735
2567 1612 1919 1748 636
2742 1648 2039 1849 666
3075 2219 2549 1885 624
3584 2565 2952 2166 736
3025 2161 2539 1891 636
3179 2283 1670 1991 668
1441 1441 1441 1441 1442
1489 1489 1489 1489 1490
1439 1439 1439 1439 1440
1441 1441 1441 1441 1442


(28, 5)

In [3]:
# Merge all the dataframes into one complete using concat function then export to CSV
# 
#
wx_type = ['air_pressure(mBar)', 
           'air_temperature', 
           'sea_water_practical_salinity', 
           'sea_water_temperature',
           'wind_speed']

c = 0
for i in range(len(sensors)):
    for j in range(len(months)):
        
        df_all = []
        wx_type_na = []
        for k in range(len(wx_type)):
      
            if len(dfx[k][c]) == 1:
                print(f'------->{sensors[i]}-{months[j]}, missing type: {wx_type[k]}')
                wx_type_na.append(wx_type[k])
            else:
                df_all.append(dfx[k][c])

        #print(len(df_all))
                
        df_all = reduce(lambda  left,right: pd.merge(left,right,on=['date'],
                                            how='outer'), df_all).dropna()
        
        print(f'{sensors[i]}-{months[j]}, records {len(df_all)}')
        
        # Remove duplicate columns
        df = df_all.T.drop_duplicates().T
        
        #create a new column for the missing weather type and populate with 9999999 as default identifier
        #Null willl need to be replaced with average values after clean dataframe
        for l in range(len(wx_type_na)):
            df.loc[:,wx_type_na[l]] = pd.Series('NULL', index=df.index)
        
        # Keep only the columns needed
        df_clean = df[['sensor_x', 'latitude_x', 'longitude_x',
           'air_pressure(mBar)', 'air_temperature',   'sea_water_practical_salinity', 'sea_water_temperature', 'wind_speed']]
        
        # Rename duplicate column headers to original
        df_clean = df_clean.rename(columns={'sensor_x':'sensor', 'latitude_x':'latitude', 'longitude_x':'longitude'})
        
        #Set the file path to export city data
        csvFile = os.path.join("../data/2017/v2",f"{sensors[i]}-{monthsf[j]}.csv")

        #Export clean datframe to csv file
        df_clean.to_csv(csvFile)
        
        print(f'Export completed csvfile {c}')
        print()
        
        #Increment counter by 1
        c+=1

------->42019-June 2017, missing type: sea_water_practical_salinity
42019-June 2017, records 587
Export completed csvfile 0

------->42019-July 2017, missing type: sea_water_practical_salinity
42019-July 2017, records 661
Export completed csvfile 1

------->42019-August 2017, missing type: sea_water_practical_salinity
42019-August 2017, records 521
Export completed csvfile 2

------->42019-Sept 2017, missing type: sea_water_practical_salinity
42019-Sept 2017, records 632
Export completed csvfile 3

42043-June 2017, records 763
Export completed csvfile 4

------->42043-July 2017, missing type: sea_water_practical_salinity
42043-July 2017, records 727
Export completed csvfile 5

42043-August 2017, records 440
Export completed csvfile 6

42043-Sept 2017, records 1440
Export completed csvfile 7

42044-June 2017, records 1440
Export completed csvfile 8

42044-July 2017, records 1485
Export completed csvfile 9

42044-August 2017, records 1488
Export completed csvfile 10

42044-Sept 2017, rec

In [4]:
df_clean

,sensor,latitude,longitude,air_pressure(mBar),air_temperature,sea_water_practical_salinity,sea_water_temperature,wind_speed
date,,,,,,,,
2017-09-01 00:00:00+00:00,ioos:station:TABS:X:1,27.066,-96.3383,1012.36,28.7,36.43,27.96,0
2017-09-01 00:30:00+00:00,ioos:station:TABS:X:1,27.066,-96.3383,1012.53,28.4,36.38,27.96,0
2017-09-01 01:00:00+00:00,ioos:station:TABS:X:1,27.066,-96.3383,1012.66,28.2,36.34,27.79,0
2017-09-01 01:30:00+00:00,ioos:station:TABS:X:1,27.066,-96.3383,1013.18,28.1,36.34,27.66,0
2017-09-01 02:00:00+00:00,ioos:station:TABS:X:1,27.066,-96.3383,1013.4,28.1,36.34,27.46,0
...,...,...,...,...,...,...,...,...
2017-09-30 21:30:00+00:00,ioos:station:TABS:X:1,27.066,-96.3383,1013.73,0,32.48,29.05,0
2017-09-30 22:00:00+00:00,ioos:station:TABS:X:1,27.066,-96.3383,1013.96,0,32.47,29.05,0
2017-09-30 22:30:00+00:00,ioos:station:TABS:X:1,27.066,-96.3383,1014.23,0,32.49,29.04,0
